In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, Window as W

In [2]:
spark = SparkSession.builder.master("local[*]").appName("Practice").getOrCreate()

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 54064)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates = read_int(self.rfile)
  File "/spark/python/pyspark/serializers.

In [3]:
spark.version

'3.5.1'

In [4]:
data = [
    # User u1 - Day 1 (two sessions)
    ("2025-08-28 09:00:00", "login",  "u1"),
    ("2025-08-28 09:15:00", "click",  "u1"),
    ("2025-08-28 09:30:00", "logout", "u1"),

    ("2025-08-28 14:00:00", "login",  "u1"),
    ("2025-08-28 14:20:00", "scroll", "u1"),
    ("2025-08-28 14:45:00", "logout", "u1"),

    # User u2 - Day 1 (single long session)
    ("2025-08-28 10:00:00", "login",  "u2"),
    ("2025-08-28 10:30:00", "purchase", "u2"),
    ("2025-08-28 11:15:00", "logout", "u2"),

    # User u3 - Day 1 (short session)
    ("2025-08-28 11:00:00", "login",  "u3"),
    ("2025-08-28 11:05:00", "logout", "u3"),

    # User u1 - Day 2
    ("2025-08-29 08:00:00", "login",  "u1"),
    ("2025-08-29 08:30:00", "click",  "u1"),
    ("2025-08-29 08:45:00", "logout", "u1"),

    # User u2 - Day 2
    ("2025-08-29 09:15:00", "login",  "u2"),
    ("2025-08-29 09:45:00", "scroll", "u2"),
    ("2025-08-29 10:15:00", "logout", "u2"),

    # User u3 - Day 2 (two sessions)
    ("2025-08-29 10:00:00", "login",  "u3"),
    ("2025-08-29 10:20:00", "logout", "u3"),

    ("2025-08-29 16:00:00", "login",  "u3"),
    ("2025-08-29 16:45:00", "logout", "u3"),
]




df = spark.createDataFrame(data, ["timestamp", "event", "user_id"])

In [5]:
df.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- event: string (nullable = true)
 |-- user_id: string (nullable = true)



In [6]:
df = df.withColumn(
    "timestamp",
    F.to_timestamp(F.col("timestamp"))
)

df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- event: string (nullable = true)
 |-- user_id: string (nullable = true)



In [7]:
df.show()

+-------------------+--------+-------+
|          timestamp|   event|user_id|
+-------------------+--------+-------+
|2025-08-28 09:00:00|   login|     u1|
|2025-08-28 09:15:00|   click|     u1|
|2025-08-28 09:30:00|  logout|     u1|
|2025-08-28 14:00:00|   login|     u1|
|2025-08-28 14:20:00|  scroll|     u1|
|2025-08-28 14:45:00|  logout|     u1|
|2025-08-28 10:00:00|   login|     u2|
|2025-08-28 10:30:00|purchase|     u2|
|2025-08-28 11:15:00|  logout|     u2|
|2025-08-28 11:00:00|   login|     u3|
|2025-08-28 11:05:00|  logout|     u3|
|2025-08-29 08:00:00|   login|     u1|
|2025-08-29 08:30:00|   click|     u1|
|2025-08-29 08:45:00|  logout|     u1|
|2025-08-29 09:15:00|   login|     u2|
|2025-08-29 09:45:00|  scroll|     u2|
|2025-08-29 10:15:00|  logout|     u2|
|2025-08-29 10:00:00|   login|     u3|
|2025-08-29 10:20:00|  logout|     u3|
|2025-08-29 16:00:00|   login|     u3|
+-------------------+--------+-------+
only showing top 20 rows



In [8]:
w = W.partitionBy("user_id").orderBy("timestamp")

In [9]:
df = df.filter((F.col("event") == "login") | (F.col("event") == "logout"))

In [10]:
df.show()

+-------------------+------+-------+
|          timestamp| event|user_id|
+-------------------+------+-------+
|2025-08-28 09:00:00| login|     u1|
|2025-08-28 09:30:00|logout|     u1|
|2025-08-28 14:00:00| login|     u1|
|2025-08-28 14:45:00|logout|     u1|
|2025-08-28 10:00:00| login|     u2|
|2025-08-28 11:15:00|logout|     u2|
|2025-08-28 11:00:00| login|     u3|
|2025-08-28 11:05:00|logout|     u3|
|2025-08-29 08:00:00| login|     u1|
|2025-08-29 08:45:00|logout|     u1|
|2025-08-29 09:15:00| login|     u2|
|2025-08-29 10:15:00|logout|     u2|
|2025-08-29 10:00:00| login|     u3|
|2025-08-29 10:20:00|logout|     u3|
|2025-08-29 16:00:00| login|     u3|
|2025-08-29 16:45:00|logout|     u3|
+-------------------+------+-------+



In [11]:
df = df.withColumn(
    "logged_out_timestamp",
    F.lead(F.col("timestamp")).over(w)
)

In [12]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- event: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- logged_out_timestamp: timestamp (nullable = true)



In [13]:
df.show()

+-------------------+------+-------+--------------------+
|          timestamp| event|user_id|logged_out_timestamp|
+-------------------+------+-------+--------------------+
|2025-08-28 09:00:00| login|     u1| 2025-08-28 09:30:00|
|2025-08-28 09:30:00|logout|     u1| 2025-08-28 14:00:00|
|2025-08-28 14:00:00| login|     u1| 2025-08-28 14:45:00|
|2025-08-28 14:45:00|logout|     u1| 2025-08-29 08:00:00|
|2025-08-29 08:00:00| login|     u1| 2025-08-29 08:45:00|
|2025-08-29 08:45:00|logout|     u1|                NULL|
|2025-08-28 10:00:00| login|     u2| 2025-08-28 11:15:00|
|2025-08-28 11:15:00|logout|     u2| 2025-08-29 09:15:00|
|2025-08-29 09:15:00| login|     u2| 2025-08-29 10:15:00|
|2025-08-29 10:15:00|logout|     u2|                NULL|
|2025-08-28 11:00:00| login|     u3| 2025-08-28 11:05:00|
|2025-08-28 11:05:00|logout|     u3| 2025-08-29 10:00:00|
|2025-08-29 10:00:00| login|     u3| 2025-08-29 10:20:00|
|2025-08-29 10:20:00|logout|     u3| 2025-08-29 16:00:00|
|2025-08-29 16

In [14]:
df = df.filter(F.col("event") == "login")

df.show()

+-------------------+-----+-------+--------------------+
|          timestamp|event|user_id|logged_out_timestamp|
+-------------------+-----+-------+--------------------+
|2025-08-28 09:00:00|login|     u1| 2025-08-28 09:30:00|
|2025-08-28 14:00:00|login|     u1| 2025-08-28 14:45:00|
|2025-08-29 08:00:00|login|     u1| 2025-08-29 08:45:00|
|2025-08-28 10:00:00|login|     u2| 2025-08-28 11:15:00|
|2025-08-29 09:15:00|login|     u2| 2025-08-29 10:15:00|
|2025-08-28 11:00:00|login|     u3| 2025-08-28 11:05:00|
|2025-08-29 10:00:00|login|     u3| 2025-08-29 10:20:00|
|2025-08-29 16:00:00|login|     u3| 2025-08-29 16:45:00|
+-------------------+-----+-------+--------------------+



In [15]:
df = df.withColumn(
    "session_time",
   F.col("timestamp") - F.col("logged_out_timestamp")
)

df.show(truncate=False)

+-------------------+-----+-------+--------------------+------------------------------------+
|timestamp          |event|user_id|logged_out_timestamp|session_time                        |
+-------------------+-----+-------+--------------------+------------------------------------+
|2025-08-28 09:00:00|login|u1     |2025-08-28 09:30:00 |INTERVAL '-0 00:30:00' DAY TO SECOND|
|2025-08-28 14:00:00|login|u1     |2025-08-28 14:45:00 |INTERVAL '-0 00:45:00' DAY TO SECOND|
|2025-08-29 08:00:00|login|u1     |2025-08-29 08:45:00 |INTERVAL '-0 00:45:00' DAY TO SECOND|
|2025-08-28 10:00:00|login|u2     |2025-08-28 11:15:00 |INTERVAL '-0 01:15:00' DAY TO SECOND|
|2025-08-29 09:15:00|login|u2     |2025-08-29 10:15:00 |INTERVAL '-0 01:00:00' DAY TO SECOND|
|2025-08-28 11:00:00|login|u3     |2025-08-28 11:05:00 |INTERVAL '-0 00:05:00' DAY TO SECOND|
|2025-08-29 10:00:00|login|u3     |2025-08-29 10:20:00 |INTERVAL '-0 00:20:00' DAY TO SECOND|
|2025-08-29 16:00:00|login|u3     |2025-08-29 16:45:00 |INTE

In [16]:
df.groupby("user_id").agg(
    F.sum("session_time").alias("total_time_spent")
).show(truncate=False)

+-------+------------------------------------+
|user_id|total_time_spent                    |
+-------+------------------------------------+
|u1     |INTERVAL '-0 02:00:00' DAY TO SECOND|
|u2     |INTERVAL '-0 02:15:00' DAY TO SECOND|
|u3     |INTERVAL '-0 01:10:00' DAY TO SECOND|
+-------+------------------------------------+



In [17]:
data = [
    # User u1 - Day 1 (two sessions)
    ("2025-08-28 09:00:00", "login",  "u1"),
    ("2025-08-28 09:15:00", "click",  "u1"),
    ("2025-08-28 09:30:00", "logout", "u1"),

    ("2025-08-28 14:00:00", "login",  "u1"),
    ("2025-08-28 14:20:00", "scroll", "u1"),
    ("2025-08-28 14:45:00", "logout", "u1"),

    # User u2 - Day 1 (single long session)
    ("2025-08-28 10:00:00", "login",  "u2"),
    ("2025-08-28 10:30:00", "purchase", "u2"),
    ("2025-08-28 11:15:00", "logout", "u2"),

    # User u3 - Day 1 (short session)
    ("2025-08-28 11:00:00", "login",  "u3"),
    ("2025-08-28 11:05:00", "logout", "u3"),

    # User u1 - Day 2
    ("2025-08-29 08:00:00", "login",  "u1"),
    ("2025-08-29 08:30:00", "click",  "u1"),
    ("2025-08-29 08:45:00", "logout", "u1"),

    # User u2 - Day 2
    ("2025-08-29 09:15:00", "login",  "u2"),
    ("2025-08-29 09:45:00", "scroll", "u2"),
    ("2025-08-29 10:15:00", "logout", "u2"),

    # User u3 - Day 2 (two sessions)
    ("2025-08-29 10:00:00", "login",  "u3"),
    ("2025-08-29 10:20:00", "logout", "u3"),

    ("2025-08-29 16:00:00", "login",  "u3"),
    ("2025-08-29 16:45:00", "logout", "u3"),
]




df = spark.createDataFrame(data, ["timestamp", "event", "user_id"]).withColumn(
    "timestamp",
    F.to_timestamp(F.col("timestamp"))
)

In [18]:
df.show()

+-------------------+--------+-------+
|          timestamp|   event|user_id|
+-------------------+--------+-------+
|2025-08-28 09:00:00|   login|     u1|
|2025-08-28 09:15:00|   click|     u1|
|2025-08-28 09:30:00|  logout|     u1|
|2025-08-28 14:00:00|   login|     u1|
|2025-08-28 14:20:00|  scroll|     u1|
|2025-08-28 14:45:00|  logout|     u1|
|2025-08-28 10:00:00|   login|     u2|
|2025-08-28 10:30:00|purchase|     u2|
|2025-08-28 11:15:00|  logout|     u2|
|2025-08-28 11:00:00|   login|     u3|
|2025-08-28 11:05:00|  logout|     u3|
|2025-08-29 08:00:00|   login|     u1|
|2025-08-29 08:30:00|   click|     u1|
|2025-08-29 08:45:00|  logout|     u1|
|2025-08-29 09:15:00|   login|     u2|
|2025-08-29 09:45:00|  scroll|     u2|
|2025-08-29 10:15:00|  logout|     u2|
|2025-08-29 10:00:00|   login|     u3|
|2025-08-29 10:20:00|  logout|     u3|
|2025-08-29 16:00:00|   login|     u3|
+-------------------+--------+-------+
only showing top 20 rows



In [19]:
df = df.withColumn(
    "date",
    F.to_date(F.col("timestamp"))
)

In [21]:
df = df.withColumn(
    "day",
    F.day(F.col("date"))
).show()

+-------------------+--------+-------+----------+---+
|          timestamp|   event|user_id|      date|day|
+-------------------+--------+-------+----------+---+
|2025-08-28 09:00:00|   login|     u1|2025-08-28| 28|
|2025-08-28 09:15:00|   click|     u1|2025-08-28| 28|
|2025-08-28 09:30:00|  logout|     u1|2025-08-28| 28|
|2025-08-28 14:00:00|   login|     u1|2025-08-28| 28|
|2025-08-28 14:20:00|  scroll|     u1|2025-08-28| 28|
|2025-08-28 14:45:00|  logout|     u1|2025-08-28| 28|
|2025-08-28 10:00:00|   login|     u2|2025-08-28| 28|
|2025-08-28 10:30:00|purchase|     u2|2025-08-28| 28|
|2025-08-28 11:15:00|  logout|     u2|2025-08-28| 28|
|2025-08-28 11:00:00|   login|     u3|2025-08-28| 28|
|2025-08-28 11:05:00|  logout|     u3|2025-08-28| 28|
|2025-08-29 08:00:00|   login|     u1|2025-08-29| 29|
|2025-08-29 08:30:00|   click|     u1|2025-08-29| 29|
|2025-08-29 08:45:00|  logout|     u1|2025-08-29| 29|
|2025-08-29 09:15:00|   login|     u2|2025-08-29| 29|
|2025-08-29 09:45:00|  scrol